In [94]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.metrics import roc_auc_score, f1_score,r2_score,classification_report,accuracy_score,precision_score, recall_score
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')
from sklearn.model_selection import KFold, cross_val_score,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler




In [2]:
def replace_target(df,name):
    df_1 = df.copy()
    d = { j:(i+1) for i,j in enumerate(iris_data['Species'].unique())}
    df_1[name] = df_1[name].map(d)
    return df_1

In [3]:
def make_cross_validation_classif_multi_n(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy,
                          params,print_is = True):
   
    
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros((X.shape[0],y.shape[1]))   

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        estimator.fit(x_train, y_train,**params) 

        y_valid_pred = estimator.predict(x_valid)
        y_train_pred = estimator.predict(x_train)
        
        
        fold_train_scores.append(metric(y_train, y_train_pred,multi_class = 'ovr'))
        fold_valid_scores.append(metric(y_valid, y_valid_pred,multi_class = 'ovr'))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        if print_is:
            print(msg)
            print("="*69)
        estimators.append(estimator)
    oof_score = metric(y, oof_predictions,multi_class = 'ovr')
    if print_is:
        print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
        print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
        print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions  

In [4]:
def enc_target(train,feature,target):
    d = train[train[target]==1].groupby(feature).size()/len(train)*100
    return d

In [5]:
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [6]:
def from_one_hot(layer):
    arr = np.argmax(layer,axis=1)+1
    return arr.reshape(-1,1)

In [100]:
# Функция для тестирования можелей неройных сетей
def train_model_result(model,data,params={'epochs':15,
                                           'batch_size':None, 
                                           'validation_split':0.2,
                                           'use_multiprocessing' : True,
                                           'verbose':0}  ):
    res= []
    import time
    start = time.time()
    hh=model.fit( data['X_train'],data['y_train'], **params)
    print('Результаты обучения модели')
    history=hh.history
    finish = time.time()
    for key in history:
        res.append((key,history[key][-1]))
        print (key,history[key][-1])
    delta_time = (finish-start)
    print('время работы-', delta_time, 'сек')
    try:
        print('Результаты валидации модели')
        val = model.evaluate( data['X_test'],data['y_test'])
        return res,val,delta_time,history
    except:
        return res,'val',delta_time,history
    finish = time.time()

In [101]:
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=100)

In [102]:
# Загрузим данные
df = pd.read_csv("train.csv")

In [103]:
df

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,Rent,402192.0,< 1 year,0.0,3.0,8.5,107866.0,0.0,NaN,0.0,other,Short Term,129360.0,73492.0,1900.0,697.0,0
7496,Home Mortgage,1533984.0,1 year,0.0,10.0,26.5,686312.0,0.0,43.0,0.0,debt consolidation,Long Term,444048.0,456399.0,12783.0,7410.0,1
7497,Rent,1878910.0,6 years,0.0,12.0,32.1,1778920.0,0.0,NaN,0.0,buy a car,Short Term,99999999.0,477812.0,12479.0,748.0,0
7498,Home Mortgage,NaN,NaN,0.0,21.0,26.5,1141250.0,0.0,NaN,0.0,debt consolidation,Short Term,615274.0,476064.0,37118.0,NaN,0


In [9]:
# Обработаем данные

df_work = df.copy()

d={}
for i in df_work['Years in current job'].value_counts().index:
    if i[:2]=='10':
        d[i]=10
        continue
    if i[0]=='<':
        d[i]=0
        continue    
df_work['Years in current job'] = df_work['Years in current job'].map(d) 


df_work.loc[df_work['Maximum Open Credit']>7000000,'Maximum Open Credit']=7000000

df_work.fillna(999,inplace=True)

target_col = ['Credit Default']
categorical_features=['Home Ownership','Tax Liens','Purpose','Term']
discrete_feature = ['Years in current job','Number of Open Accounts',\
                    'Years of Credit History','Number of Credit Problems',\
                   'Months since last delinquent','Bankruptcies','Credit Score']
continuous_feature = ['Annual Income','Maximum Open Credit','Current Loan Amount',\
                      'Current Credit Balance','Monthly Debt']

target = 'Credit Default'

for i in categorical_features:
    d = enc_target(df_work,i,target)
    df_work[i] = df_work[i].map(d)
    df_work[i] = df_work[[i]].fillna(0)

    
X_train =  df_work.drop('Credit Default',1 )
y_train = df_work[target]
# y_train= y_train.map({0:1,1:2})

for i in categorical_features:
    X_train=pd.concat([X_train,pd.get_dummies(X_train[i],prefix=i)],axis=1)
    X_train.drop(i,1,inplace=True)
    
for i in continuous_feature + discrete_feature:
    X_train[i]=StandardScaler().fit_transform(X_train[[i]])    
    

In [10]:
# нормализуем данные
X_train= normalize(X_train.to_numpy())
y_train= to_categorical(y_train.to_numpy())

In [95]:
ind,ind_test = train_test_split(a, train_size=0.8)
X,Y,x,y = X_train[ind],y_train[ind],X_train[ind_test],y_train[ind_test]

In [107]:
X.shape,Y.shape,x.shape,y.shape

((6000, 37), (6000, 2), (1500, 37), (1500, 2))

### Тестируем несколько слоем при одинаковом количестве нейронов

In [96]:
# Создадим 12 моделей с разным количством слоев с 1000 нейронами
n=1000
finish_n = y_train.shape[1]
models = []
input_dim = X_train.shape[1]
hide_levels = [2,3,5,10]
for level in hide_levels:
    n_nueron = (n-finish_n)//(level-1)
    lost_nueron = (n-finish_n)%(level-1)
    if level>2:
        levels_list = [n_nueron+lost_nueron]+[n_nueron for i in range(level-2)]+[finish_n]
    else:
        levels_list =[n-finish_n,finish_n]    
    for fun in['tanh','sigmoid','relu']:
        model_name = f'{level}_level_{"_".join([str(i) for i in levels_list])}_nueron_{fun}'
        denses_list = [Dense(levels_list[0],activation=fun, input_dim=input_dim,name='1')]
        if level>2:
            denses_list+=[ Dense(i,activation=fun,name = str(k+2)) for k,i in enumerate(levels_list[1:-1])] 
        denses_list+=[ Dense(levels_list[-1],activation='softmax',name = str(level))]  
        my_model = Sequential(denses_list,name =model_name )
        models.append((my_model,my_model.get_weights()))
        print(model_name)
print('количесвто моделей - ',len(models))

2_level_998_2_nueron_tanh
2_level_998_2_nueron_sigmoid
2_level_998_2_nueron_relu
3_level_499_499_2_nueron_tanh
3_level_499_499_2_nueron_sigmoid
3_level_499_499_2_nueron_relu
5_level_251_249_249_249_2_nueron_tanh
5_level_251_249_249_249_2_nueron_sigmoid
5_level_251_249_249_249_2_nueron_relu
10_level_118_110_110_110_110_110_110_110_110_2_nueron_tanh
10_level_118_110_110_110_110_110_110_110_110_2_nueron_sigmoid
10_level_118_110_110_110_110_110_110_110_110_2_nueron_relu
количесвто моделей -  12


In [97]:
for model in models:
    model[0].compile(
          optimizer='adam',
          loss='categorical_crossentropy',
          metrics=[keras.metrics.AUC()],
                    )

In [108]:
# Параметры для fit
params={'epochs':15,
       'batch_size':None, 
       'validation_split':0.2,
       'use_multiprocessing' : True,
       'verbose':0} 
data = {'X_train':X,
        'y_train':Y,
        'X_test':x,
       'y_test':y}

In [125]:
# тестировать будем на кросвалидации. делим на 5 фолдоф, оцениваем по OOF-score по всей выборке
# make_cross_validation_classif_multi_n(X_train, y_train,models[0][0],\
#                                 roc_auc_score,cv_strategy,print_is =True,params=params)[1]

In [109]:
# Запускаем тестирование созданых моделей с разными эпохами
result=pd.DataFrame(columns=['model','epoch','result_valid'])
for epoh in [15,30,50,100,300]:
    params['epochs']=epoh
    for model in models:
        model[0].set_weights(model[1])
        answer = train_model_result(model[0], data,params)[1]
        result = result.append({'model':model[0].name,'epoch':epoh,'result_valid':answer},ignore_index=True)

Результаты обучения модели
loss 0.4692268967628479
auc_188 0.8485201597213745
val_loss 0.4501694142818451
val_auc_188 0.863937497138977
время работы- 2.66542387008667 сек
Результаты валидации модели
47/47 [==============================] - 0s 618us/step - loss: 0.4748 - auc_188: 0.8456
Результаты обучения модели
loss 0.47345465421676636
auc_189 0.8465317487716675
val_loss 0.45579415559768677
val_auc_189 0.8649864196777344
время работы- 3.1979100704193115 сек
Результаты валидации модели
47/47 [==============================] - 0s 788us/step - loss: 0.4814 - auc_189: 0.8474
Результаты обучения модели
loss 0.42726632952690125
auc_190 0.8787396550178528
val_loss 0.46321573853492737
val_auc_190 0.8537513613700867
время работы- 3.2089200019836426 сек
Результаты валидации модели
47/47 [==============================] - 0s 767us/step - loss: 0.4854 - auc_190: 0.8408
Результаты обучения модели
loss 0.47082218527793884
auc_191 0.84784996509552
val_loss 0.45802026987075806
val_auc_191 0.859807968

47/47 [==============================] - 0s 980us/step - loss: 0.4743 - auc_192: 0.8467
Результаты обучения модели
loss 0.1244514212012291
auc_193 0.9924653768539429
val_loss 1.133418321609497
val_auc_193 0.779874324798584
время работы- 14.149873971939087 сек
Результаты валидации модели
47/47 [==============================] - 0s 958us/step - loss: 1.2309 - auc_193: 0.7796
Результаты обучения модели
loss 0.46034708619117737
auc_194 0.8556817770004272
val_loss 0.45565682649612427
val_auc_194 0.8588979244232178
время работы- 16.824305772781372 сек
Результаты валидации модели
47/47 [==============================] - 0s 958us/step - loss: 0.4737 - auc_194: 0.8482
Результаты обучения модели
loss 0.46629080176353455
auc_195 0.8480094075202942
val_loss 0.44918152689933777
val_auc_195 0.865094780921936
время работы- 16.824307918548584 сек
Результаты валидации модели
47/47 [==============================] - 0s 958us/step - loss: 0.4708 - auc_195: 0.8488
Результаты обучения модели
loss 0.0844800

47/47 [==============================] - 0s 745us/step - loss: 3.6262 - auc_196: 0.7510
Результаты обучения модели
loss 0.42424532771110535
auc_197 0.8818491697311401
val_loss 0.5011488795280457
val_auc_197 0.8409292101860046
время работы- 66.40241432189941 сек
Результаты валидации модели
47/47 [==============================] - 0s 639us/step - loss: 0.5327 - auc_197: 0.8288
Результаты обучения модели
loss 0.596072793006897
auc_198 0.7150583267211914
val_loss 0.5914506912231445
val_auc_198 0.721666693687439
время работы- 67.08103036880493 сек
Результаты валидации модели
47/47 [==============================] - 0s 639us/step - loss: 0.5930 - auc_198: 0.7200
Результаты обучения модели
loss 0.01153797097504139
auc_199 0.9997493028640747
val_loss 2.317877769470215
val_auc_199 0.7535319328308105
время работы- 66.86283278465271 сек
Результаты валидации модели
47/47 [==============================] - 0s 618us/step - loss: 2.4798 - auc_199: 0.7498


In [144]:
result.sort_values('result_valid',ascending=False,inplace=True,key = lambda x :x.apply(lambda y: y[1]))
result

,model,epoch,result_valid
43,5_level_251_249_249_249_2_nueron_sigmoid,100,"[0.4715786576271057, 0.8499224781990051]"
21,10_level_118_110_110_110_110_110_110_110_110_2...,30,"[0.4709087610244751, 0.8492146730422974]"
7,5_level_251_249_249_249_2_nueron_sigmoid,15,"[0.47682884335517883, 0.8490848541259766]"
31,5_level_251_249_249_249_2_nueron_sigmoid,50,"[0.47080716490745544, 0.8487746119499207]"
42,5_level_251_249_249_249_2_nueron_tanh,100,"[0.4733791649341583, 0.8485720157623291]"
30,5_level_251_249_249_249_2_nueron_tanh,50,"[0.47372210025787354, 0.8482499718666077]"
4,3_level_499_499_2_nueron_sigmoid,15,"[0.47304973006248474, 0.8479944467544556]"
6,5_level_251_249_249_249_2_nueron_tanh,15,"[0.4728965759277344, 0.8477962017059326]"
19,5_level_251_249_249_249_2_nueron_sigmoid,30,"[0.4789752662181854, 0.8476300239562988]"
1,2_level_998_2_nueron_sigmoid,15,"[0.48140329122543335, 0.8473848700523376]"


### Тестируем двухслойные модели с разным количеством нейронов

In [111]:
%%time
N=1000
result2=pd.DataFrame()
for n_nueron in range(10,N+1,50):
    for fun in['tanh','sigmoid','relu']:
        dens =[ Dense(n_nueron,input_dim=X_train.shape[1],activation=fun,name= '1'),
                Dense(y_train.shape[1],activation='softmax',name='exit')]
        model_level_2 = Sequential(dens, name=f'{n_nueron}_{fun}')
        model_level_2.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                               metrics=[keras.metrics.AUC()])
        a = model_level_2.get_weights()   
                
        for epoh in [15,30,100]:
            model_level_2.set_weights(a)    
            params['epochs']=epoh
            answer = train_model_result(model_level_2, data,params)[1]
            result2 = result2.append({'model':model_level_2.name,'epoch':epoh,'result_valid':answer},ignore_index=True)    
                
                

Результаты обучения модели
loss 0.4665260314941406
auc_200 0.8506477475166321
val_loss 0.44938716292381287
val_auc_200 0.865203857421875
время работы- 1.8176538944244385 сек
Результаты валидации модели
47/47 [==============================] - 0s 511us/step - loss: 0.4728 - auc_200: 0.8470
Результаты обучения модели
loss 0.4654637575149536
auc_200 0.8510956764221191
val_loss 0.45047906041145325
val_auc_200 0.864464282989502
время работы- 2.5823490619659424 сек
Результаты валидации модели
47/47 [==============================] - 0s 490us/step - loss: 0.4738 - auc_200: 0.8457
Результаты обучения модели
loss 0.4627145826816559
auc_200 0.852762758731842
val_loss 0.45037445425987244
val_auc_200 0.864608645439148
время работы- 8.49072551727295 сек
Результаты валидации модели
47/47 [==============================] - 0s 469us/step - loss: 0.4755 - auc_200: 0.8457
Результаты обучения модели
loss 0.47732463479042053
auc_201 0.849889874458313
val_loss 0.45803114771842957
val_auc_201 0.866673946380

47/47 [==============================] - 0s 511us/step - loss: 0.5186 - auc_208: 0.8358
Результаты обучения модели
loss 0.46788743138313293
auc_209 0.8491219282150269
val_loss 0.45206940174102783
val_auc_209 0.8634825944900513
время работы- 1.9767985343933105 сек
Результаты валидации модели
47/47 [==============================] - 0s 532us/step - loss: 0.4736 - auc_209: 0.8461
Результаты обучения модели
loss 0.46791112422943115
auc_209 0.8490985631942749
val_loss 0.45040538907051086
val_auc_209 0.8644055128097534
время работы- 2.9226596355438232 сек
Результаты валидации модели
47/47 [==============================] - 0s 511us/step - loss: 0.4731 - auc_209: 0.8468
Результаты обучения модели
loss 0.4662677049636841
auc_209 0.850334644317627
val_loss 0.4493092894554138
val_auc_209 0.8649718761444092
время работы- 9.612745523452759 сек
Результаты валидации модели
47/47 [==============================] - 0s 490us/step - loss: 0.4757 - auc_209: 0.8466
Результаты обучения модели
loss 0.467855

47/47 [==============================] - 0s 532us/step - loss: 0.4904 - auc_217: 0.8465
Результаты обучения модели
loss 0.3343239724636078
auc_217 0.932295024394989
val_loss 0.5145739912986755
val_auc_217 0.83369380235672
время работы- 10.876896381378174 сек
Результаты валидации модели
47/47 [==============================] - 0s 554us/step - loss: 0.5707 - auc_217: 0.8170
Результаты обучения модели
loss 0.4687069058418274
auc_218 0.8490179181098938
val_loss 0.44792333245277405
val_auc_218 0.8656447529792786
время работы- 2.1699748039245605 сек
Результаты валидации модели
47/47 [==============================] - 0s 532us/step - loss: 0.4738 - auc_218: 0.8465
Результаты обучения модели
loss 0.467581570148468
auc_218 0.8493331670761108
val_loss 0.44815123081207275
val_auc_218 0.8658878803253174
время работы- 3.341040849685669 сек
Результаты валидации модели
47/47 [==============================] - 0s 554us/step - loss: 0.4735 - auc_218: 0.8472
Результаты обучения модели
loss 0.46656173467

47/47 [==============================] - 0s 575us/step - loss: 0.5975 - auc_226: 0.8246
Результаты обучения модели
loss 0.467263787984848
auc_227 0.8496344089508057
val_loss 0.4510713219642639
val_auc_227 0.8639784455299377
время работы- 2.314105272293091 сек
Результаты валидации модели
47/47 [==============================] - 0s 575us/step - loss: 0.4766 - auc_227: 0.8465
Результаты обучения модели
loss 0.467256098985672
auc_227 0.8495875597000122
val_loss 0.4500751197338104
val_auc_227 0.8640507459640503
время работы- 3.5962724685668945 сек
Результаты валидации модели
47/47 [==============================] - 0s 554us/step - loss: 0.4737 - auc_227: 0.8465
Результаты обучения модели
loss 0.46623465418815613
auc_227 0.8502581119537354
val_loss 0.44984909892082214
val_auc_227 0.8651933670043945
время работы- 11.928853511810303 сек
Результаты валидации модели
47/47 [==============================] - 0s 575us/step - loss: 0.4750 - auc_227: 0.8468
Результаты обучения модели
loss 0.475206494

Результаты обучения модели
loss 0.46839192509651184
auc_236 0.8490167260169983
val_loss 0.4580104947090149
val_auc_236 0.86417156457901
время работы- 3.512195587158203 сек
Результаты валидации модели
47/47 [==============================] - 0s 554us/step - loss: 0.4783 - auc_236: 0.8451
Результаты обучения модели
loss 0.46731576323509216
auc_236 0.849529504776001
val_loss 0.4515536427497864
val_auc_236 0.8635402917861938
время работы- 11.529489755630493 сек
Результаты валидации модели
47/47 [==============================] - 0s 575us/step - loss: 0.4759 - auc_236: 0.8462
Результаты обучения модели
loss 0.475675106048584
auc_237 0.8445112705230713
val_loss 0.44942134618759155
val_auc_237 0.8648045063018799
время работы- 2.9256627559661865 сек
Результаты валидации модели
47/47 [==============================] - 0s 554us/step - loss: 0.4727 - auc_237: 0.8467
Результаты обучения модели
loss 0.4704442322254181
auc_237 0.8480403423309326
val_loss 0.45080941915512085
val_auc_237 0.86468887329

47/47 [==============================] - 0s 618us/step - loss: 0.4750 - auc_245: 0.8465
Результаты обучения модели
loss 0.4680027663707733
auc_245 0.8491268157958984
val_loss 0.44891658425331116
val_auc_245 0.8660115003585815
время работы- 3.8515045642852783 сек
Результаты валидации модели
47/47 [==============================] - 0s 596us/step - loss: 0.4740 - auc_245: 0.8461
Результаты обучения модели
loss 0.4669578969478607
auc_245 0.8496174216270447
val_loss 0.44985494017601013
val_auc_245 0.863902747631073
время работы- 12.760610103607178 сек
Результаты валидации модели
47/47 [==============================] - 0s 596us/step - loss: 0.4778 - auc_245: 0.8459
Результаты обучения модели
loss 0.4705471694469452
auc_246 0.8482774496078491
val_loss 0.45090389251708984
val_auc_246 0.8656094074249268
время работы- 2.41019344329834 сек
Результаты валидации модели
47/47 [==============================] - 0s 596us/step - loss: 0.4739 - auc_246: 0.8457
Результаты обучения модели
loss 0.47150275

47/47 [==============================] - 0s 596us/step - loss: 0.5070 - auc_253: 0.8368
Результаты обучения модели
loss 0.249225914478302
auc_253 0.966586172580719
val_loss 0.6141362190246582
val_auc_253 0.8313406109809875
время работы- 12.498370885848999 сек
Результаты валидации модели
47/47 [==============================] - 0s 575us/step - loss: 0.6934 - auc_253: 0.8177
Результаты обучения модели
loss 0.46974822878837585
auc_254 0.8479284048080444
val_loss 0.45301932096481323
val_auc_254 0.8642809391021729
время работы- 2.3891761302948 сек
Результаты валидации модели
47/47 [==============================] - 0s 575us/step - loss: 0.4753 - auc_254: 0.8448
Результаты обучения модели
loss 0.4681503176689148
auc_254 0.8494192957878113
val_loss 0.4487822949886322
val_auc_254 0.8651197552680969
время работы- 3.785444498062134 сек
Результаты валидации модели
47/47 [==============================] - 0s 618us/step - loss: 0.4748 - auc_254: 0.8463
Результаты обучения модели
loss 0.466197550296

In [145]:
result2.sort_values('result_valid',ascending=False,inplace=True,key = lambda x :x.apply(lambda y: y[1]))
result2

,epoch,model,result_valid
3,15.0,10_sigmoid,"[0.47799429297447205, 0.8491649031639099]"
12,15.0,60_sigmoid,"[0.47165796160697937, 0.8486364483833313]"
4,30.0,10_sigmoid,"[0.4722318947315216, 0.8483333587646484]"
7,30.0,10_relu,"[0.47014155983924866, 0.8481511473655701]"
24,15.0,110_relu,"[0.47311922907829285, 0.8480611443519592]"
...,...,...,...
116,100.0,610_relu,"[0.6303102374076843, 0.8070182800292969]"
170,100.0,910_relu,"[0.6725055575370789, 0.8062602281570435]"
98,100.0,510_relu,"[0.6283459067344666, 0.8037912845611572]"
134,100.0,710_relu,"[0.6660685539245605, 0.8009037971496582]"


### Тестируем с разными оптимизаторам

In [113]:
ful_result=pd.DataFrame()
for fun in['sigmoid','relu']:
    dens =[ Dense(910,input_dim=X_train.shape[1],activation=fun,name= '1'),
            Dense(y_train.shape[1],activation='softmax',name='exit')]
    model = Sequential(dens, name=f'960_{fun}')
    a = model.get_weights()
    for epoh in [15,30,100]:
        params['epochs'] = epoh
        for opt in [keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Nadam()]:       
                model.compile(optimizer=opt,
                              loss='categorical_crossentropy',
                              metrics=[keras.metrics.AUC()])
                model.set_weights(a) 
                answer = train_model_result(model, data,params)[1]
                ful_result = ful_result.append({'model':model.name,\
                                             'epoch':epoh,
                                              'opt':opt,  
                                              'result_valid':answer},ignore_index=True)    

Результаты обучения модели
loss 0.4909948706626892
auc_260 0.8374741673469543
val_loss 0.5121831297874451
val_auc_260 0.8642936944961548
время работы- 2.608372926712036 сек
Результаты валидации модели
47/47 [==============================] - 0s 596us/step - loss: 0.5412 - auc_260: 0.8464
Результаты обучения модели
loss 0.584078848361969
auc_261 0.7547582387924194
val_loss 0.5687565207481384
val_auc_261 0.8299708366394043
время работы- 2.2470452785491943 сек
Результаты валидации модели
47/47 [==============================] - 0s 575us/step - loss: 0.5723 - auc_261: 0.8165
Результаты обучения модели
loss 0.4741533398628235
auc_262 0.8459030389785767
val_loss 0.4493306875228882
val_auc_262 0.865487813949585
время работы- 2.956688404083252 сек
Результаты валидации модели
47/47 [==============================] - 0s 596us/step - loss: 0.4733 - auc_262: 0.8475
Результаты обучения модели
loss 0.48633331060409546
auc_263 0.8398560881614685
val_loss 0.5240570902824402
val_auc_263 0.8620728850364

In [146]:
ful_result.sort_values('result_valid',ascending=False,inplace=True,key = lambda x :x.apply(lambda y: y[1]))
ful_result

,epoch,model,opt,result_valid
13,30.0,960_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.47515058517456055, 0.8492520451545715]"
16,100.0,960_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.46958184242248535, 0.8491426706314087]"
7,100.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.49156153202056885, 0.8487106561660767]"
2,15.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.47325843572616577, 0.8474612832069397]"
0,15.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5412423610687256, 0.8463671207427979]"
6,100.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.47887951135635376, 0.8463495373725891]"
5,30.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.4799065887928009, 0.846327543258667]"
9,15.0,960_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.4824109971523285, 0.8462457060813904]"
3,30.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5545619130134583, 0.84451824426651]"
11,15.0,960_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.4840054512023926, 0.8444468379020691]"


### Тестируем 10 слойную сеть с разными оптимизаторами

In [115]:
ful_result2=pd.DataFrame()
for fun in['sigmoid','relu']:
    dens =[ Dense(100,input_dim=X_train.shape[1],activation=fun,name= '1'),
            Dense(100,activation=fun,name= '2'),
           Dense(100,activation=fun,name= '3'),
           Dense(100,activation=fun,name= '4'),
           Dense(100,activation=fun,name= '5'),
           Dense(100,activation=fun,name= '6'),
           Dense(100,activation=fun,name= '7'),
           Dense(100,activation=fun,name= '8'),
           Dense(100,activation=fun,name= '9'),
            Dense(y_train.shape[1],activation='softmax',name='exit')]
    model = Sequential(dens, name=f'10_100_{fun}')
    a = model.get_weights()
    for epoh in [15,30,100]:
        params['epochs'] = epoh
        for opt in [keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Nadam()]:       
                model.compile(optimizer=opt,
                              loss='categorical_crossentropy',
                              metrics=[keras.metrics.AUC()])
                model.set_weights(a) 
                answer = train_model_result(model, data,params)[1]
                ful_result2 = ful_result2.append({'model':model.name,\
                                             'epoch':epoh,
                                              'opt':opt,  
                                              'result_valid':answer},ignore_index=True)    

Результаты обучения модели
loss 0.5978651642799377
auc_278 0.7173083424568176
val_loss 0.5984219908714294
val_auc_278 0.721666693687439
время работы- 6.911288499832153 сек
Результаты валидации модели
47/47 [==============================] - 0s 958us/step - loss: 0.6005 - auc_278: 0.7200
Результаты обучения модели
loss 0.597247302532196
auc_279 0.7122902870178223
val_loss 0.5914989113807678
val_auc_279 0.721666693687439
время работы- 4.4070093631744385 сек
Результаты валидации модели
47/47 [==============================] - 0s 937us/step - loss: 0.5930 - auc_279: 0.7200
Результаты обучения модели
loss 0.5975351333618164
auc_280 0.7149513959884644
val_loss 0.5913732647895813
val_auc_280 0.721666693687439
время работы- 9.247413396835327 сек
Результаты валидации модели
47/47 [==============================] - 0s 1ms/step - loss: 0.5930 - auc_280: 0.7200
Результаты обучения модели
loss 0.5975884795188904
auc_281 0.7179416418075562
val_loss 0.5918760299682617
val_auc_281 0.721666693687439
вр

In [148]:
ful_result2.sort_values('result_valid',ascending=False,inplace=True,key = lambda x :x.apply(lambda y: y[1]))
ful_result2

,epoch,model,opt,result_valid
13,30.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.4728471636772156, 0.848301351070404]"
16,100.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.5167338848114014, 0.8402437567710876]"
9,15.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5056843161582947, 0.8373461961746216]"
11,15.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.5282987356185913, 0.824054479598999]"
10,15.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.5466872453689575, 0.8171062469482422]"
12,30.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.9817948341369629, 0.8130910992622375]"
14,30.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[1.0072762966156006, 0.7838453650474548]"
15,100.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[9.171859741210938, 0.7805795073509216]"
17,100.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[2.8246028423309326, 0.7688429355621338]"
3,30.0,10_100_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5935824513435364, 0.7200000286102295]"


### Тестируем 3 слойную сеть с разными оптимизаторами

In [117]:
%%time
ful_result3=pd.DataFrame()
for fun in['sigmoid','relu']:
    dens =[ Dense(499,input_dim=X_train.shape[1],activation=fun,name= '1'),
            Dense(499,activation=fun,name= '2'),
            Dense(y_train.shape[1],activation='softmax',name='exit')]
    model = Sequential(dens, name=f'499_499_10_{fun}')
    a = model.get_weights()
    for epoh in [30,100,200]:
        params['epochs'] = epoh
        for opt in [keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Nadam(),'adam']:       
                model.compile(optimizer=opt,
                              loss='categorical_crossentropy',
                              metrics=[keras.metrics.AUC()])
                model.set_weights(a) 
                answer =train_model_result(model, data,params)[1]
                ful_result3 = ful_result3.append({'model':model.name,\
                                             'epoch':epoh,
                                              'opt':opt,  
                                              'result_valid':answer},ignore_index=True)    

Результаты обучения модели
loss 0.4670107960700989
auc_296 0.8475953340530396
val_loss 0.45297694206237793
val_auc_296 0.8656718730926514
время работы- 11.53749680519104 сек
Результаты валидации модели
47/47 [==============================] - 0s 1ms/step - loss: 0.4690 - auc_296: 0.8496
Результаты обучения модели
loss 0.5980185270309448
auc_297 0.7245424389839172
val_loss 0.5872401595115662
val_auc_297 0.8261653184890747
время работы- 9.117295980453491 сек
Результаты валидации модели
47/47 [==============================] - 0s 1ms/step - loss: 0.5895 - auc_297: 0.8116
Результаты обучения модели
loss 0.4667541980743408
auc_298 0.8493382930755615
val_loss 0.45136749744415283
val_auc_298 0.8658427000045776
время работы- 13.742503643035889 сек
Результаты валидации модели
47/47 [==============================] - 0s 1ms/step - loss: 0.4702 - auc_298: 0.8479
Результаты обучения модели
loss 0.46777984499931335
auc_299 0.8489771485328674
val_loss 0.45675933361053467
val_auc_299 0.86588335037231

In [149]:
ful_result3.sort_values('result_valid',ascending=False,inplace=True,key = lambda x :x.apply(lambda y: y[1]))
ful_result3

,epoch,model,opt,result_valid
13,30.0,499_499_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.4716598391532898, 0.8504271507263184]"
6,100.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.4715076982975006, 0.8500231504440308]"
7,100.0,499_499_10_sigmoid,adam,"[0.47152361273765564, 0.8498998284339905]"
0,30.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.4690430164337158, 0.8496149182319641]"
9,200.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.4811727702617645, 0.8487193584442139]"
10,200.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.49792149662971497, 0.8485866785049438]"
17,100.0,499_499_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.47051867842674255, 0.848372220993042]"
8,200.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5013988614082336, 0.8483672738075256]"
3,30.0,499_499_10_sigmoid,adam,"[0.47328898310661316, 0.8482300043106079]"
2,30.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.47016558051109314, 0.8479459881782532]"


## Результаты

In [150]:
result.head(10)

,model,epoch,result_valid
43,5_level_251_249_249_249_2_nueron_sigmoid,100,"[0.4715786576271057, 0.8499224781990051]"
21,10_level_118_110_110_110_110_110_110_110_110_2...,30,"[0.4709087610244751, 0.8492146730422974]"
7,5_level_251_249_249_249_2_nueron_sigmoid,15,"[0.47682884335517883, 0.8490848541259766]"
31,5_level_251_249_249_249_2_nueron_sigmoid,50,"[0.47080716490745544, 0.8487746119499207]"
42,5_level_251_249_249_249_2_nueron_tanh,100,"[0.4733791649341583, 0.8485720157623291]"
30,5_level_251_249_249_249_2_nueron_tanh,50,"[0.47372210025787354, 0.8482499718666077]"
4,3_level_499_499_2_nueron_sigmoid,15,"[0.47304973006248474, 0.8479944467544556]"
6,5_level_251_249_249_249_2_nueron_tanh,15,"[0.4728965759277344, 0.8477962017059326]"
19,5_level_251_249_249_249_2_nueron_sigmoid,30,"[0.4789752662181854, 0.8476300239562988]"
1,2_level_998_2_nueron_sigmoid,15,"[0.48140329122543335, 0.8473848700523376]"


In [152]:
result2.head(10)

,epoch,model,result_valid
3,15.0,10_sigmoid,"[0.47799429297447205, 0.8491649031639099]"
12,15.0,60_sigmoid,"[0.47165796160697937, 0.8486364483833313]"
4,30.0,10_sigmoid,"[0.4722318947315216, 0.8483333587646484]"
7,30.0,10_relu,"[0.47014155983924866, 0.8481511473655701]"
24,15.0,110_relu,"[0.47311922907829285, 0.8480611443519592]"
51,15.0,260_relu,"[0.4737386107444763, 0.8480424284934998]"
114,15.0,610_relu,"[0.48112964630126953, 0.8479457497596741]"
21,15.0,110_sigmoid,"[0.471890389919281, 0.8479037880897522]"
15,15.0,60_relu,"[0.4750930070877075, 0.8478913903236389]"
96,15.0,510_relu,"[0.4771116077899933, 0.8478315472602844]"


In [153]:
ful_result.head(10)

,epoch,model,opt,result_valid
13,30.0,960_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.47515058517456055, 0.8492520451545715]"
16,100.0,960_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.46958184242248535, 0.8491426706314087]"
7,100.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.49156153202056885, 0.8487106561660767]"
2,15.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.47325843572616577, 0.8474612832069397]"
0,15.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5412423610687256, 0.8463671207427979]"
6,100.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.47887951135635376, 0.8463495373725891]"
5,30.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.4799065887928009, 0.846327543258667]"
9,15.0,960_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.4824109971523285, 0.8462457060813904]"
3,30.0,960_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5545619130134583, 0.84451824426651]"
11,15.0,960_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.4840054512023926, 0.8444468379020691]"


In [154]:
ful_result2.head(10)

,epoch,model,opt,result_valid
13,30.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.4728471636772156, 0.848301351070404]"
16,100.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.5167338848114014, 0.8402437567710876]"
9,15.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5056843161582947, 0.8373461961746216]"
11,15.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.5282987356185913, 0.824054479598999]"
10,15.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.5466872453689575, 0.8171062469482422]"
12,30.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.9817948341369629, 0.8130910992622375]"
14,30.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[1.0072762966156006, 0.7838453650474548]"
15,100.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[9.171859741210938, 0.7805795073509216]"
17,100.0,10_100_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[2.8246028423309326, 0.7688429355621338]"
3,30.0,10_100_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5935824513435364, 0.7200000286102295]"


In [155]:
ful_result3.head(10)

,epoch,model,opt,result_valid
13,30.0,499_499_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.4716598391532898, 0.8504271507263184]"
6,100.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.4715076982975006, 0.8500231504440308]"
7,100.0,499_499_10_sigmoid,adam,"[0.47152361273765564, 0.8498998284339905]"
0,30.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.4690430164337158, 0.8496149182319641]"
9,200.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.4811727702617645, 0.8487193584442139]"
10,200.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.49792149662971497, 0.8485866785049438]"
17,100.0,499_499_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,"[0.47051867842674255, 0.848372220993042]"
8,200.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,"[0.5013988614082336, 0.8483672738075256]"
3,30.0,499_499_10_sigmoid,adam,"[0.47328898310661316, 0.8482300043106079]"
2,30.0,499_499_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,"[0.47016558051109314, 0.8479459881782532]"


In [158]:
ful_result3.loc[13]

epoch                                                        30.0
model                                             499_499_10_relu
opt             <tensorflow.python.keras.optimizer_v2.gradient...
result_valid             [0.4716598391532898, 0.8504271507263184]
Name: 13, dtype: object

Победитель -   499_499_10_relu	SGD	0.879445  , проверим его результат 0.8504271507263184

In [163]:
params['epochs'] = 30
dens =[ Dense(499,input_dim=X_train.shape[1],activation='relu',name= '1'),
        Dense(499,activation='relu',name= '2'),
        Dense(y_train.shape[1],activation='softmax',name='exit')]
model = Sequential(dens, name=f'499_499_2_relu')
model.compile(optimizer=keras.optimizers.SGD(),
            loss='categorical_crossentropy',
            metrics=[keras.metrics.AUC()])
model.set_weights(a) 
model.fit(X,Y,**params)

In [167]:
# model.fit(X,Y,**params)

In [168]:
# Замер на отложеннй выборке
roc_auc_score(y,model.predict(x))

0.7544378306878307

Какая то фигня получилась

Почему то реальная валидация и валидация Кераса разные результаты дают